In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W44`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W45`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W46`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W47`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W48`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W49`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W50`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W51`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W52`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W53`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W01`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W02`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
52031,2021-01-07 08:02:59+00:00,2021-01-07 08:03:08+00:00,Production,0001 of 0003,U2,None,None,KL7,Friday Assembly,Thursday Morning,iXUsr_Sianen(10382),4,0.15,2021-02
51993,2021-01-07 08:07:16+00:00,2021-01-07 08:08:53+00:00,Production,0003 of 0003,U2,None,None,KL7,Friday Assembly,Thursday Morning,iXUsr_Sianen(10382),4,1.62,2021-02
51967,2021-01-07 08:08:53+00:00,2021-01-07 08:17:15+00:00,Preparation/Changeover,None,U2,None,None,KL7,None,Thursday Morning,iXUsr_Sianen(10382),4,8.37,2021-02
52177,2021-01-10 06:28:57+00:00,2021-01-10 06:29:09+00:00,Production,0001 of 0011,H2,None,None,KL7,Monday Assembly,Sunday Morning,iXUsr_Sianen(10382),3,0.20,2021-02
0,2021-01-07 09:15:24+00:00,2021-01-07 15:51:32+00:00,Break,None,J2,move to other line,None,KL8,None,Thursday Afternoon,Ala,0,396.13,2021-02


In [5]:
data_in_use.shape

(70755, 14)